문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
1. 데이터 로드 및 전처리
2. 채널별 CAC 계산
    - 2024년 데이터만 사용
    - 각 채널별 총 마케팅 비용 집계
    - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
    - CAC = 총 마케팅 비용 / 신규 고객 수
    - organic과 referral 채널은 CAC = 0으로 처리
3. 고객별 LTV 계산
4. 채널별 LTV 및 ROI 분석
    - 채널별 평균 LTV 계산
    - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
    - payback period = CAC / (월평균 구매금액) 계산
5. 시각화 및 전략 제안 (15분)
    - CAC vs LTV 산점도 차트
    - 채널별 ROI 막대 차트
    - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)


문제 2: 고객 코호트 분석 및 유지율 개선 전략
목표: "월별 가입 코호트"의 유지율과 매출 기여도를 분석하여 고객 유지 전략을 수립하세요.
1. 코호트 그룹 생성
2. 월별 유지율 계산
    - 각 거래의 코호트 대비 경과 개월 수 계산
    - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
    - 유지율 테이블 생성 (코호트 × 경과월)
    - 평균 유지율 곡선 계산
3. 코호트별 매출 기여도 분석
    - 각 코호트의 월별 누적 매출 계산
    - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
    - 6개월 후, 12개월 후 예상 LTV 추정
4. 유지율 히트맵 시각화
5. 개선 전략 수립
    - 유지율이 급격히 감소하는 시점 식별
    - 고성과 코호트 vs 저성과 코호트 특성 비교
    - 유지율 개선을 위한 액션 아이템 3가지 제안

In [55]:
#파이썬으로 파일 불러오기 
with open('./customer_satisfaction.csv', encoding='EUC-KR') as f:
    print(f.readline())

import pandas as pd

# encoding은 파일 불러올때 매우 중요
# df = pd.read_csv('./namsan.csv', encoding='EUC-KR', low_memory=False)
df = pd.read_csv('./customer_satisfaction.csv', encoding='EUC-KR')

customer_id,contact_date,contact_reason,satisfaction_score,resolution_time_hours,repeat_contact



In [56]:
import pandas as pd
import numpy as np

# 각 파일을 개별 DataFrame으로 불러오기
df1 = pd.read_csv('/Users/gim-yujin/Desktop/TIL/데이터분석 /customer_satisfaction.csv')
df2 = pd.read_csv('/Users/gim-yujin/Desktop/TIL/데이터분석 /customer_transactions.csv')
df3 = pd.read_csv('/Users/gim-yujin/Desktop/TIL/데이터분석 /marketing_performance.csv')



In [57]:
#데이터 살펴보기 연락날짜, 연락목적, 만족도, 해결까지 걸린시간, 재연락여부
satisfaction = df1 
satisfaction 
df1.isna().sum()

customer_id              0
contact_date             0
contact_reason           0
satisfaction_score       0
resolution_time_hours    0
repeat_contact           0
dtype: int64

In [58]:
#가입일, 고객유입채널, 고객 등급, 거래 날짜, 주문 금액, 상품카테고리 
transcation = df2 
transcation
df2.isna().sum()

customer_id            0
registration_date      0
acquisition_channel    0
customer_segment       0
transaction_date       0
order_value            0
product_category       0
dtype: int64

In [59]:
# 마케팅 채널, 마케팅비용, 신규고객, 홈페이지 방문수, 전환
perfotmance = df3
perfotmance
df3.isna().sum()



month              0
channel            0
marketing_spend    0
new_customers      0
website_visits     0
conversions        0
dtype: int64

In [ ]:
# 문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
# 목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
# 1. 데이터 로드 및 전처리
customer = pd.merge(df1, df2, on='customer_id')
df_sorted = customer.sort_values(by='customer_id', ascending=True)

#채널별 CAC 계산:고객 1명을 유치하기 위해 들어간 마케팅/세일즈 비용 총비용/신규고객수 

# 2. 채널별 CAC 계산
#     - 2024년 데이터만 사용
df3['month'] = pd.to_datetime(df3['month'])
df_2024 = df3[df3['month'].dt.year == 2024]
#     - 각 채널별 총 마케팅 비용 집계
total_spend = df_2024.groupby('channel')['marketing_spend'].sum()
#     - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
total_new = df_2024.groupby('channel')['new_customers'].sum() 
#     - CAC = 총 마케팅 비용 / 신규 고객 수 
df_2024['cac'] = df_2024['marketing_spend'] / df_2024['new_customers']
df_2024
#     - organic과 referral 채널은 CAC = 0으로 처리
df_2024['cac'] = np.where(
    (df_2024['channel'].isin(['organic', 'referral'])) & (df_2024['cac'].isna()),
    0,
    df_2024['cac']
)
df_2024
#'month'를 인덱스로, 'channel'을 열로, 'marketing_spend'를 값으로 pivot
df_pivot = df_2024.pivot_table(index='month', columns='channel', values='cac', aggfunc='sum')
df_pivot
#3. 고객별 LTV 계산(한명의 고객이 회사에 기여하는 총 수익)고객별 총 주문 금액의 합 
df2['ltv_customer'] = df2.groupby('customer_id')['order_value'].transform('sum')
ltv_customer =  df2.groupby('customer_id')['order_value'].sum()
# 4. 채널별 LTV 및 ROI 분석(두개의 데이터 프레임 결합 )
customer_channel = df2[['customer_id', 'acquisition_channel']].drop_duplicates()
ltv_by_channel_df = pd.merge(ltv_customer, customer_channel, on='customer_id', how='left')
print(ltv_by_channel_df)

#     - 채널별 평균 LTV 계산
#     - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
#     - payback period = CAC / (월평균 구매금액) 계산
# 5. 시각화 및 전략 제안 (15분)
#     - CAC vs LTV 산점도 차트
#     - 채널별 ROI 막대 차트
#     - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)


     customer_id  order_value acquisition_channel
0              1       938.00               email
1              2       660.75         paid_search
2              3       548.71         paid_search
3              4       102.23        social_media
4              5       378.85               email
..           ...          ...                 ...
995          996        44.13             organic
996          997        88.83        social_media
997          998       217.17            referral
998          999       208.73        social_media
999         1000        57.47            referral

[1000 rows x 3 columns]


/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_28225/332496063.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['cac'] = df_2024['marketing_spend'] / df_2024['new_customers']
/var/folders/98/vj3q9b254k79y681ng12dddh0000gn/T/ipykernel_28225/332496063.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024['cac'] = np.where(


In [89]:
df2


,customer_id,registration_date,acquisition_channel,customer_segment,transaction_date,order_value,product_category,ltv_customer
0,1,2023-04-13,email,premium,2023-08-22,104.24,Books,938.00
1,1,2023-04-13,email,premium,2024-01-15,141.21,Electronics,938.00
2,1,2023-04-13,email,premium,2024-02-21,54.80,Electronics,938.00
3,1,2023-04-13,email,premium,2023-06-11,159.59,Home,938.00
4,1,2023-04-13,email,premium,2023-09-30,111.29,Fashion,938.00
...,...,...,...,...,...,...,...,...
4564,999,2023-06-04,social_media,basic,2023-06-26,49.34,Books,208.73
4565,999,2023-06-04,social_media,basic,2024-01-11,34.43,Fashion,208.73
4566,999,2023-06-04,social_media,basic,2023-08-01,41.79,Fashion,208.73
4567,1000,2023-03-17,referral,basic,2023-06-04,29.25,Books,57.47


In [61]:
df_2024

,month,channel,marketing_spend,new_customers,website_visits,conversions,cac
60,2024-01-01,organic,0.000000,18,471,18,0.0
61,2024-01-01,paid_search,8760.203892,18,991,18,NaN
62,2024-01-01,social_media,5272.708407,15,992,15,NaN
63,2024-01-01,email,1957.893749,12,1050,12,NaN
64,2024-01-01,referral,1288.390831,6,983,6,0.0
65,2024-02-01,organic,0.000000,21,501,21,0.0
66,2024-02-01,paid_search,6758.754072,9,996,9,NaN
67,2024-02-01,social_media,5096.866032,12,1000,12,NaN
68,2024-02-01,email,2053.619904,10,997,10,NaN
69,2024-02-01,referral,349.933305,9,961,9,0.0
